# 采集多副静电势的数据

In [82]:
import bempp.api
import numpy as np
import meshio
import pygmsh as pg

grid = bempp.api.import_grid('Blade_new.msh')

dp0_space = bempp.api.function_space(grid, "DP", 0)
p1_space = bempp.api.function_space(grid, "P", 1)

identity = bempp.api.operators.boundary.sparse.identity(
    p1_space, p1_space, dp0_space)
dlp = bempp.api.operators.boundary.laplace.double_layer(
    p1_space, p1_space, dp0_space)
slp = bempp.api.operators.boundary.laplace.single_layer(
    dp0_space, p1_space, dp0_space)


data = []

RF峰值电压设置为300V,sin函数扫描

In [83]:
i = 0
def dirichlet_data(x, n, domain_index, result):
#     print(domain_index)
#     result[0] = 1./(4 * np.pi * ((x[0] - 1.5)**2 + x[1]**2 + x[2]**2)**(0.5))
    global i
    if domain_index in [1]:      # 在不同区域设置相应的电压
        result[0] = 300.0 * np.sin(np.pi / 6 * i)
  #         print(domain_index)
    elif domain_index in [2]:
         result[0] = 0.0
    else:
         result[0] = 0.0
 #        print('other')
 #        print(domain_index)
    i += 1

    


In [86]:
def data_multy():
    halfrange = 10
    dirichlet_fun = bempp.api.GridFunction(p1_space, fun=dirichlet_data)
    rhs = (-.5 * identity + dlp) * dirichlet_fun
    neumann_fun, info = bempp.api.linalg.cg(slp, rhs, tol=1E-3)
    n_grid_points = 1000
    plot_grid = np.mgrid[-10000:10000:n_grid_points*1j, -10000:10000:n_grid_points*1j]
    points = np.vstack((plot_grid[0].ravel(),
                        plot_grid[1].ravel(),
                        np.zeros(plot_grid[0].size)))
    slp_pot = bempp.api.operators.potential.laplace.single_layer(
        dp0_space, points)
    dlp_pot = bempp.api.operators.potential.laplace.double_layer(
        p1_space, points)
    u_evaluated = slp_pot * neumann_fun - dlp_pot * dirichlet_fun
    u_evaluated = u_evaluated.reshape((n_grid_points,n_grid_points))
    radius = np.sqrt(plot_grid[0]**2 + plot_grid[1]**2)
    # //  but not / ,ensure n_grid_points//2 is int type
    data_single = u_evaluated[n_grid_points//2-halfrange:n_grid_points//2+halfrange,n_grid_points//2-halfrange:n_grid_points//2+halfrange]
    data.append(data_single)

In [85]:
%%time

# pickle the data of electric potential for generate movie
# range() is the number of picture and every picture interval 30 degree

for i in range(72):
    data_multy()
    
import pickle

f = open('data_new', 'wb')
pickle.dump(data, f)
f.close()

CPU times: user 2d 2h 47min 1s, sys: 13min 23s, total: 2d 3h 24s
Wall time: 7h 42min 13s
